# Sample Foam Plant Planning

## Problem Statement
- Foam plants have complex scheduling process
- Mould arrangement on foram production line (racetracks) have numerous constraints which represent a large number of possible solutions

#### Products
- For this POC, we are considering 3 products: __Prod1, Prod2 and Prod3__

#### Production Plan/Customer Demand
- Demand is defined by Weeks based on the customer demand
- This POC will be consider optimization for a __6 week period. Customer Demand and Profit__

| Week | Prod1 | Prod2 | Prod3 |
| --- | --- | --- | --- |
| PROFIT | 10 | 6 | 8 |
| Week1 | 5000 | 6000 | 3000 |
| Week2 | 6000 | 5000 | 2000 |
| Week3 | 3000 | 3000 | 2000 |
| Week4 | 2000 | 3000 | 4000 |
| Week5 | 2000 | 1000 | 5000 |
| Week6 | 5000 | 5000 | 1000 |

#### Moulds
- Moulds are Tools or Cavity used to create the foam seats. There are __10 moulds__ available of __3 types__
- A product can be made from only certain mould types. Below is the mapping and the time required in minutes:
- __Asuumption is that it takes 40 minutes to mount Moulds onto a cassette TBD__

| Mould-Type | Prod1 | Prod2 | Prod3 |
| --- | --- | --- | --- |
| Mould-TypeA | 7 | 0 | 0 |
| Mould-TypeB | 0 | 7 | 0 |
| Mould-TypeC | 0 | 0 | 7 |

- __Total moulds: 20__
- Mould-TypeA - 7 quantities
- Mould-TypeB - 7 quantities
- Mould-TypeC - 6 quantities

#### Cassette - TBD
- Cassette is a container on which Moulds are placed. Cassettes are placed on a carrier once Moulds are placed on them. Prior to mounting Moulds, Cassettes need to be prepared. When the cassette is ready with proper Moulds, Line stops and old cassette is demounted from carrier. New cassette with new tools is mounted on freed carrier.
- __Assumption__ is that any mould can be mounted on any cassette. There are __15 cassetes__

#### Carriers - TBD
- Carriers are fixed on the racetrack. It moves around the racetrack at a constant speed. 
- __Assumption__ here is that there are a total of __10 carriers__ on the racetrack

#### Production Line/Racetrack - TBD
- Production Lines are called __Racetracks__. Line speed is the speed at carriers go around the racetrack
- __Assumption__ is __Line Speed = 5 carriers/hour__

#### Factory Schedule
- Factory runs 7 days a week, 3 shifts a day of 8 hours each
- __Breaks in eah shift:__ 10 min, 20 min and 10 min
- __Production Time in each shift__ = 480 mins - 40 mins = 440 mins

- __Assumption for periods in each shift: TBD__
    - Each shift has 8 hours = __480 mins__
    
    - __Period 1: 140 mins__ = 2 hours, 20 mins
    - __Break  1:  10 mins__
    
    - __Period 2: 160 mins__ = 2 hours, 40 mins
    - __Break  2:  20 mins__
    
    - __Period 3: 140 mins__ = 2 hours, 20 mins
    - __Break 3:   10 mins__

- __Changeover  TBD__
- This is when a cassette is demounted and a new cassette is loaded
- Each 10 min break can have maximum of 1 changeover
- Each 20 mins break can have a maximum of 2 changeovers

#### Inventory
- Upto 500 units of each product may be stored in inventory per week at a cost of $0.50 per unit per week.
- At the start of Week1, there is no product inventory
- By end of Week6, there should be 1000 units of each product in Inventory

#### Other constraints out of scope for this POC
- Cassette to Line compatibility
- Tool to Line compatibility
- Tool to Carrier compatibility
- Tilt
- Vaccum not more than 1 in 3
- Holiday schedule
- Human effort involved
- Human schedule

### Objectives
- How can we achieve maximum profit
- What is the most optimized schedule
- Does the most optimized schedule give us the least number of changovers?

### Modelling

#### Sets and Indices

t ∊ Weeks = {Week1, Week2, Week3, Week4, Week5, Week6}

p ∊ Products = {Prod1, Prod2, Prod3}

m ∊ Moulds = {Mould-TypeA, Mould-TypeB, Mould-TypeC}

#### Parameters

- hours_per_week -> Tme (in mins/week) available for Production of the products on a weekly basis = Number of working days in a week (7) * number of shifts (3) * duration of a shift (8) MINUS the break periods
- max_inventory -> Maximum number of units of a single product type that can be stored in inventory at any given month
- holding_cost -> monthly cost in (USD/unit/month) of keeping in inventory a unit of any product type
- store_target -> Number of units of each product type to keep in inventory at the end of a planning phase

- profit(p) -> Proft (in USD/unit) of product p
- installed(m) -> Number of moulds of type m installed in the factory
- max_sales(t,p) -> Number of units of product p that can be sold in week t
- time_req (m,p) -> Time (in mins/unit) needed on a mould m to manufacture one unit of product p

#### Decision Variables

- make(t,p) -> Number of units of product p to manufacture in week t
- store(t,p)[0, max_inventory] -> Number of units of product p to store in week t
- sell(t,p)[0, max_sales(t,p)] -> Number of units of product p to sell in month t

#### Constraints

- __Initial Balance:__ For each product p, the number of units produced should be equal to the number of units sold plus the number of units stored. Here since Week1 is used since it is the first week

        make(Week1,p) = sell(t,p) + store(Week1,p)
        
- __Balance:__ For each product p, the number of units produced in a week t and previously stored should be equal to the number of units sold and stored in that month
    
        store(t-1, p) + make(t,p) = sell(t,p) + store(t,p)

#### Objective
- Maximize profit
- Optimize schedule

### Implementation

In [31]:
import numpy as np
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

In [32]:
# Sets
products = ['Prod1', 'Prod2', 'Prod3']
machines = ['Mould-TypeA', 'Mould-TypeB', 'Mould-TypeC']
weeks = ['Week1', 'Week2', 'Week3', 'Week4', 'Week5', 'Week6']
profit = {'Prod1':10, 'Prod2':6, 'Prod3':8}
time_req = {
    'Mould-TypeA': {'Prod1': 7},
    'Mould-TypeB': {'Prod2': 7},
    'Mould-TypeC': {'Prod3': 7}
}
# Number of each Moulds available assuming 
# all moulds can be used during the 
# entire production time without downtime
installed = {'Mould-TypeA': 7, 'Mould-TypeB': 7, 'Mould-TypeC': 6}

In [33]:
# Maximum Sales Forecast/Requirement
max_sales = {
    ('Week1', 'Prod1'): 5000,
    ('Week1', 'Prod2'): 6000,
    ('Week1', 'Prod3'): 3000,
    ('Week2', 'Prod1'): 6000,
    ('Week2', 'Prod2'): 5000,
    ('Week2', 'Prod3'): 2000,
    ('Week3', 'Prod1'): 3000,
    ('Week3', 'Prod2'): 3000,
    ('Week3', 'Prod3'): 2000,
    ('Week4', 'Prod1'): 2000,
    ('Week4', 'Prod2'): 3000,
    ('Week4', 'Prod3'): 4000,
    ('Week5', 'Prod1'): 2000,
    ('Week5', 'Prod2'): 1000,
    ('Week5', 'Prod3'): 5000,
    ('Week6', 'Prod1'): 5000,
    ('Week6', 'Prod2'): 5000,
    ('Week6', 'Prod3'): 1000,
}

In [34]:
holding_cost = 0.5
max_inventory = 500
store_target = 100
mins_per_week = 7*3*440

#### Model Deployment

In [35]:
# Create the Model
factory = gp.Model('Factory Planning')

# Add the Variables

# Qty manufactured
make = factory.addVars(weeks, products, name='Make')

# Quantity stored
store = factory.addVars(weeks, products, ub=max_inventory, name='Store')

# Quantity Sold
sell = factory.addVars(weeks, products, ub=max_sales, name='Sell')

In [36]:
# Add the constraints
# Initial Balance
initial_balance = factory.addConstrs((
    make[weeks[0], product] == 
    sell[weeks[0], product] + store[weeks[0], product] 
    for product in products), name='Initial_Balance')

# Balance
balance = factory.addConstrs((
    store[weeks[weeks.index(week)-1], product] + make[week, product] ==
    sell[week, product] + store[week, product]
    for product in products for week in weeks
    if week != weeks[0]), name='Balance')

In [37]:
target_inventory = factory.addConstrs((
    store[weeks[-1], product] == store_target
    for product in products), name='End_Balance')

In [38]:
# Define the objective
obj = gp.quicksum(profit[product] * sell[week, product] - 
                 holding_cost * store[week, product]
                 for week in weeks for product in products)

factory.setObjective(obj, GRB.MAXIMIZE)

In [39]:
factory.optimize()

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 21 rows, 54 columns and 72 nonzeros
Model fingerprint: 0x19cd64fa
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-01, 1e+01]
  Bounds range     [5e+02, 6e+03]
  RHS range        [1e+02, 1e+02]
Presolve removed 21 rows and 54 columns
Presolve time: 0.02s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.0385000e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds
Optimal objective  5.038500000e+05


#### Production Plan

In [40]:
rows = weeks.copy()
columns = products.copy()
make_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)
for week, product in make.keys():
    if(abs(make[week, product].x) > 1e-6):
        make_plan.loc[week, product] = np.round(make[week, product].x, 1)
make_plan

,Prod1,Prod2,Prod3
Week1,5000.0,6000.0,3000.0
Week2,6000.0,5000.0,2000.0
Week3,3000.0,3000.0,2000.0
Week4,2000.0,3000.0,4000.0
Week5,2000.0,1000.0,5000.0
Week6,5100.0,5100.0,1100.0


#### Sales Plan

In [41]:
rows = weeks.copy()
columns = products.copy()
sell_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)
for week, product in sell.keys():
    if (abs(sell[week, product].x) > 1e-6):
        sell_plan.loc[week, product] = np.round(sell[week, product].x, 1)
sell_plan

,Prod1,Prod2,Prod3
Week1,5000.0,6000.0,3000.0
Week2,6000.0,5000.0,2000.0
Week3,3000.0,3000.0,2000.0
Week4,2000.0,3000.0,4000.0
Week5,2000.0,1000.0,5000.0
Week6,5000.0,5000.0,1000.0


#### Inventory Plan

In [42]:
rows = weeks.copy()
columns = products.copy()
store_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for week, product in store.keys():
    if (abs(store[week, product].x) > 1e-6):
        store_plan.loc[week, product] = np.round(store[week, product].x, 1)
store_plan

,Prod1,Prod2,Prod3
Week1,0.0,0.0,0.0
Week2,0.0,0.0,0.0
Week3,0.0,0.0,0.0
Week4,0.0,0.0,0.0
Week5,0.0,0.0,0.0
Week6,100.0,100.0,100.0
